In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

In [13]:
data = pd.read_csv("stroke_zad7.csv", delimiter=";")
df= data.copy()
print(df.isnull().sum())
print(df.dtypes)
df.head()

gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64
gender               object
age                  object
hypertension          int64
heart_disease         int64
ever_married         object
work_type            object
Residence_type       object
avg_glucose_level    object
bmi                  object
smoking_status       object
stroke                int64
dtype: object


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67,0,1,Yes,Private,Urban,"228,69","36,6",formerly smoked,1
1,Female,61,0,0,Yes,Self-employed,Rural,"202,21","34,55",never smoked,1
2,Male,80,0,1,Yes,Private,Rural,"105,92","32,5",never smoked,1
3,Female,49,0,0,Yes,Private,Urban,"171,23","34,4",smokes,1
4,Female,79,1,0,Yes,Self-employed,Rural,"174,12",24,never smoked,1


In [14]:
def czy_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

df.loc[df['age'].apply(czy_float)==False, "age"] = np.nan
df['age'] = df['age'].astype('float')
df['avg_glucose_level'] = df['avg_glucose_level'].str.replace(',', '.')
df['avg_glucose_level'] = df['avg_glucose_level'].astype('float')
df['bmi'] = df['bmi'].str.replace(',', '.')
df['bmi'] = df['bmi'].astype('float')
df= df.dropna()
print(df.isnull().sum())

gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64


In [15]:
df = pd.get_dummies(df, drop_first=True)
df.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Male,ever_married_Yes,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,0,1,228.69,36.60,1,True,True,False,True,False,False,True,True,False,False
1,61.0,0,0,202.21,34.55,1,False,True,False,False,True,False,False,False,True,False
2,80.0,0,1,105.92,32.50,1,True,True,False,True,False,False,False,False,True,False
3,49.0,0,0,171.23,34.40,1,False,True,False,True,False,False,True,False,False,True
4,79.0,1,0,174.12,24.00,1,False,True,False,False,True,False,False,False,True,False


In [16]:
df_clean = df.copy()

In [17]:
X = df_clean.drop(columns='stroke')
y = df_clean['stroke']

imbalance_ratio = y.value_counts(normalize=True)
print('Stopień niezbalansowania:')
print(imbalance_ratio)

Stopień niezbalansowania:
stroke
0    0.95034
1    0.04966
Name: proportion, dtype: float64


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Zbiór treningowy: {X_train.shape[0]}, Zbiór testowy: {X_test.shape[0]}")

Zbiór treningowy: 3995, Zbiór testowy: 999


**budowanie modelu bez resamplingu**

In [22]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print('Model bez resamplingu')
print(classification_report(y_test, y_pred))

Model bez resamplingu
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       949
           1       0.00      0.00      0.00        50

    accuracy                           0.95       999
   macro avg       0.47      0.50      0.49       999
weighted avg       0.90      0.95      0.92       999



**oversampling**

In [24]:
smote=SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
clf_smote = RandomForestClassifier(random_state=42)
clf_smote.fit(X_train_smote, y_train_smote)
y_pred_smote = clf_smote.predict(X_test)

print('SMOTE oversampling')
print(classification_report(y_test, y_pred_smote))

SMOTE oversampling
              precision    recall  f1-score   support

           0       0.95      0.96      0.96       949
           1       0.10      0.08      0.09        50

    accuracy                           0.92       999
   macro avg       0.52      0.52      0.52       999
weighted avg       0.91      0.92      0.91       999



**undersampling**

In [25]:
undersampler = RandomUnderSampler(random_state=42)
X_train_under, y_train_under = undersampler.fit_resample(X_train, y_train)
clf_under = RandomForestClassifier(random_state=42)
clf_under.fit(X_train_under, y_train_under)
y_pred_under = clf_under.predict(X_test)

print('Wyniki oversamplingu:')
print(classification_report(y_test, y_pred_under))

Wyniki oversamplingu:
              precision    recall  f1-score   support

           0       0.98      0.69      0.81       949
           1       0.12      0.80      0.21        50

    accuracy                           0.70       999
   macro avg       0.55      0.75      0.51       999
weighted avg       0.94      0.70      0.78       999

